In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog

In [ ]:
def open_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    df = pd.read_excel(file_path, None)
    sheet_dict = {}
    for i, j in enumerate(df.keys()):
        print(i+1, j)
        sheet_dict.setdefault(i+1, j)
    ans = eval(input('Please select the sheet:'))
    df = df[sheet_dict[ans]]
    return df

In [ ]:
revenue = open_file()

In [ ]:
budget = open_file()

In [ ]:
revenue = revenue[(revenue['BG'] != 'Others') & (revenue['狀態'].str.contains('已|未')) & (revenue['預交日期'] >= '2021-01-01')]

In [ ]:
budget = budget[['負責業務', 'BG', 'Group', '預交年份', '預交月份', '數量', '本國幣別NTD', '品名', '類型']]

In [ ]:
budget = budget[(budget['預交年份'] == 2021)]

In [ ]:
budget['分類'] = '預算'

In [ ]:
revenue['類型'] = revenue.apply(lambda x: '天線' if x['BG'] == 'RF' else '元件', axis=1)

In [ ]:
revenue['分類'] = '實績'

In [ ]:
revenue = revenue[['負責業務', 'BG', 'Group', '預交年份', '預交月份', '數量', '本國幣別NTD', '品名', '類型', '分類']]

In [ ]:
table = pd.concat([revenue, budget], axis=0)

In [ ]:
table['預交月份'] = pd.Categorical(table['預交月份'], ordered=True, categories=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
       'September', 'October', 'November', 'December'])

In [ ]:
x = pd.DataFrame(table.groupby(['負責業務', '類型', '分類', '預交月份'])[['本國幣別NTD']].sum().unstack('預交月份').to_records())

In [ ]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [ ]:
x.columns = x.columns.str.replace("\('本國幣別NTD', ", "").str.replace("''", ")").str.replace(')', '').str.replace("'", "")

In [ ]:
import plotly.express as px

In [ ]:
table

In [ ]:
budget_rev = px.histogram(table, x='預交月份', y='本國幣別NTD', color='分類', facet_col='負責業務', category_orders={'預交月份': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
       'September', 'October', 'November', 'December']}, facet_row='類型', barmode='group', log_y=True, title='各區實績預算類別表')

In [ ]:
px.scatter(table.query('類型 == "天線"'), x='本國幣別NTD', y='數量', size='本國幣別NTD', color='Group', facet_col='負責業務')

In [ ]:
pd.read_clipboard()